In [72]:
import pandas as pd
import numpy as np
import rdflib
import urllib.parse
import time
from SPARQLWrapper import SPARQLWrapper, JSON  #https://rdflib.dev/sparqlwrapper/

### Reading Iconclass vocabulary

In [64]:
vocabularies = pd.read_excel("../data/FoodOn_concepts.ods", engine="odf")
vocabularies.rename(columns={'Food Concept':'Iconclass'}, inplace=True)
vocabularies['Iconclass'] = vocabularies['Iconclass'].apply(str.title)
vocabularies.head(10)

,Iconclass
0,Worcestershire Sauce
1,Cream Cheese
2,Mixture
3,Walnuts
4,Onion
5,Firm
6,Mozzarella Cheese
7,Mayonnaise
8,Artichoke
9,Tortilla


### Building a function to query SPARQL endpoints

In [73]:
def query_sparql_endpoint(endpoint, query, term):
    term = urllib.parse.quote(term)
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query%term)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return(results["results"]["bindings"])

In [75]:
term

<function str.replace(old, new, count=-1, /)>

### Querying Getty ATT Vocabulary

In [62]:
vocabularies['Getty-ATT'] = np.empty((len(vocabularies), 0)).tolist()
endpoint = "http://vocab.getty.edu/sparql"
query = '''
    SELECT ?Subject ?Term ?Parents ?Descr ?ScopeNote ?Type (coalesce(?Type1,?Type2) as ?ExtraType) {
  ?Subject luc:term "%s"; a ?typ.
  ?typ rdfs:subClassOf gvp:Subject; rdfs:label ?Type.
  FILTER (?typ != gvp:Subject)
  OPTIONAL {?Subject gvp:placeTypePreferred [gvp:prefLabelGVP [xl:literalForm ?Type1]]}
  OPTIONAL {?Subject gvp:agentTypePreferred [gvp:prefLabelGVP [xl:literalForm ?Type2]]}
  OPTIONAL {?Subject gvp:prefLabelGVP [xl:literalForm ?Term]}
  OPTIONAL {?Subject gvp:parentStringAbbrev ?Parents}
  OPTIONAL {?Subject foaf:focus/gvp:biographyPreferred/schema:description ?Descr}
  OPTIONAL {?Subject skos:scopeNote [dct:language gvp_lang:en; rdf:value ?ScopeNote]}}
'''

for idx in vocabularies.index[0:5]:
    term = vocabularies.loc[idx,'Iconclass']
    results = query_sparql_endpoint(endpoint, query, term)
    for result in results:
        #print(result["Term"]["value"])
        vocabularies.loc[idx, 'Getty-ATT'].append(result["Term"]["value"])

In [76]:
vocabularies['DBPedia'] = np.empty((len(vocabularies), 0)).tolist()
endpoint = 'http://dbpedia.org/sparql'
query = '''
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/%s> rdfs:label ?label }
'''

for idx in vocabularies.index[0:5]:
    term = vocabularies.loc[idx,'Iconclass']
    results = query_sparql_endpoint(endpoint, query, term)
    for result in results:
        if result["label"]["xml:lang"] == 'en':
            vocabularies.loc[idx, 'DBPedia'].append(result["label"]["value"])

{'label': {'type': 'literal', 'xml:lang': 'en', 'value': 'Mixture'}}
{'label': {'type': 'literal', 'xml:lang': 'ar', 'value': 'مزيج'}}
{'label': {'type': 'literal', 'xml:lang': 'de', 'value': 'Gemisch'}}
{'label': {'type': 'literal', 'xml:lang': 'es', 'value': 'Mezcla'}}
{'label': {'type': 'literal', 'xml:lang': 'fr', 'value': 'Mélange'}}
{'label': {'type': 'literal', 'xml:lang': 'it', 'value': 'Miscela'}}
{'label': {'type': 'literal', 'xml:lang': 'ja', 'value': '混合物'}}
{'label': {'type': 'literal', 'xml:lang': 'nl', 'value': 'Mengsel'}}
{'label': {'type': 'literal', 'xml:lang': 'pl', 'value': 'Mieszanina'}}
{'label': {'type': 'literal', 'xml:lang': 'pt', 'value': 'Mistura'}}
{'label': {'type': 'literal', 'xml:lang': 'ru', 'value': 'Смесь (химия)'}}
{'label': {'type': 'literal', 'xml:lang': 'zh', 'value': '混合物'}}
{'label': {'type': 'literal', 'xml:lang': 'en', 'value': 'Walnuts'}}
{'label': {'type': 'literal', 'xml:lang': 'en', 'value': 'Onion'}}
{'label': {'type': 'literal', 'xml:lang

In [63]:
vocabularies.head(10)

,Iconclass,Getty-ATT
0,worcestershire sauce,"[Worcestershire Beacon, Worcestershire, El Sau..."
1,cream cheese,"[Cheese, Chloe, Cheese, Bernard, cheese guillo..."
2,mixture,"[Mixture Bayou, Mixture Lake, color mixture, s..."
3,walnuts,"[The Walnuts, walnuts (nuts), Mnemiopsis (genus)]"
4,onion,"[Onion Creek, Onion, Bayou, Onion Creek, Onion..."
5,firm,[]
6,mozzarella cheese,[]
7,mayonnaise,[]
8,artichoke,[]
9,tortilla,[]


In [77]:
vocabularies.head(10)

,Iconclass,DBPedia
0,Worcestershire Sauce,[]
1,Cream Cheese,[]
2,Mixture,[Mixture]
3,Walnuts,[Walnuts]
4,Onion,[Onion]
5,Firm,[]
6,Mozzarella Cheese,[]
7,Mayonnaise,[]
8,Artichoke,[]
9,Tortilla,[]
